In [7]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [31]:
import pathlib
import tensorflow

print('Current path: ')
print(pathlib.Path().absolute())

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num CPUs Available: ", len(tensorflow.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Current path: 
d:\WeiKi\Desktop\FruitsAndVegetables
Num CPUs Available:  1
Num GPUs Available:  0


In [9]:
train_path = "./input/fruit-and-vegetable-image-recognition/train"
validation_path = "./input/fruit-and-vegetable-image-recognition/validation"
test_path  = "./input/fruit-and-vegetable-image-recognition/test"

categories = os.listdir(train_path)
categories.sort()
"Categories Count :" ,len(categories)

('Categories Count :', 36)

In [10]:
train = []

IMG_SIZE=100

for category in categories:
    folder = os.path.join(train_path,category)
    label = categories.index(category)
    
    for file in os.listdir(folder):
        file = os.path.join(folder,file)
        img = cv2.imread(file)
        #if type(img) == "NoneType":
        #print(file)
        
        try:
            img_arr = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            train.append([img_arr,label])
            
        except:
            pass
        
            
"Training Image Count",len(train)

('Training Image Count', 3114)

In [11]:
validation = []

count = int(0)
for category in categories:
    folder = os.path.join(validation_path,category)
    label = categories.index(category)
    
    for file in os.listdir(folder):
        file = os.path.join(folder,file)
        #print(file)
        img = cv2.imread(file)
        try:
            img_arr = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            validation.append([img_arr,label])
        except:
            count+=1
            pass
len(validation),count

(351, 0)

In [12]:
test = []
for category in categories:
    folder = os.path.join(test_path,category)
    label = categories.index(category)
    
    for file in os.listdir(folder):
        file = os.path.join(folder,file)
        img = cv2.imread(file)
        try:
            img_arr = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            test.append([img_arr,label])
        except:
            pass
len(test)

359

In [13]:
"Train:",len(train),"Validation:",len(validation),"Test :",len(test)

('Train:', 3114, 'Validation:', 351, 'Test :', 359)

In [14]:
x_train = []
y_train = []
x_validation = []
y_validation = []
x_test = []
y_test = []

for features , label  in train:
    x_train.append(features)
    y_train.append(label)
y_train = pd.get_dummies(y_train)
for features , label  in validation:
    x_validation.append(features)
    y_validation.append(label)
y_validation = pd.get_dummies(y_validation)
for features , label  in test:
    x_test.append(features)
    y_test.append(label)
y_test = pd.get_dummies(y_test)


x_train = np.array(x_train)/255
y_train = np.array(y_train)

x_validation = np.array(x_validation)/255
y_validation = np.array(y_validation)

x_test = np.array(x_test)/255
y_test = np.array(y_test)

len(x_train),len(y_train),len(x_validation),len(y_validation),len(x_test),len(y_test)

(3114, 3114, 351, 351, 359, 359)

In [15]:
y_train.shape,y_validation.shape,y_test.shape

((3114, 36), (351, 36), (359, 36))

In [36]:
print(x_train.shape)

(3114, 100, 100, 3)


# Create Model

In [16]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Activation,BatchNormalization,Dropout


In [17]:
model = Sequential()

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(3,3))

model.add(Conv2D(128,(5,5),activation="relu"))
model.add(MaxPooling2D(5,5)) 


model.add(Flatten())

#-------------------

model.add(Dense(128,input_shape = x_train.shape[1:],activation="relu"))
model.add(Dense(36,activation="softmax"))

In [32]:
model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy"])
history = model.fit(x_train,y_train,epochs=15,validation_data=(x_test,y_test))

Epoch 1/15
98/98 [==============================] - 146s 1s/step - loss: 2.5593 - accuracy: 0.2585 - val_loss: 2.0776 - val_accuracy: 0.3733
Epoch 2/15
98/98 [==============================] - 132s 1s/step - loss: 2.2216 - accuracy: 0.3372 - val_loss: 1.7226 - val_accuracy: 0.4596
Epoch 3/15
98/98 [==============================] - 128s 1s/step - loss: 2.0169 - accuracy: 0.3979 - val_loss: 1.4620 - val_accuracy: 0.5599
Epoch 4/15
98/98 [==============================] - 127s 1s/step - loss: 1.8595 - accuracy: 0.4441 - val_loss: 1.2223 - val_accuracy: 0.6602
Epoch 5/15
98/98 [==============================] - 125s 1s/step - loss: 1.6706 - accuracy: 0.4900 - val_loss: 1.1453 - val_accuracy: 0.6630
Epoch 6/15
98/98 [==============================] - 127s 1s/step - loss: 1.5487 - accuracy: 0.5244 - val_loss: 1.0905 - val_accuracy: 0.6741
Epoch 7/15
98/98 [==============================] - 126s 1s/step - loss: 1.4314 - accuracy: 0.5697 - val_loss: 1.2059 - val_accuracy: 0.6184
Epoch 8/15
98

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 128)       3584      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 128)       409728    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3200)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [35]:
import matplotlib.pyplot as plt
model.save('./keras_model.h5')